In [ ]:
!wget 'http://snap.stanford.edu/deepnetbio-ismb/ipynb/yeast.edgelist'

--2020-07-19 13:25:36--  http://snap.stanford.edu/deepnetbio-ismb/ipynb/yeast.edgelist
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11758544 (11M) [text/plain]
Saving to: ‘yeast.edgelist’

yeast.edgelist      100%[===================>]  11.21M  3.24MB/s    in 4.0s    

2020-07-19 13:25:41 (2.84 MB/s) - ‘yeast.edgelist’ saved [11758544/11758544]



In [ ]:
import networkx as nx
import scipy.sparse as sp

import torch
import torch.nn.modules.loss
import torch.nn.functional as F

import argparse
import time

import networkx as nx
import scipy.sparse as sp

import torch
import torch.nn.modules.loss
import torch.nn as nn
import torch.nn.functional as F


from torch import optim
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter

import numpy as np

In [ ]:
!pip show torch

Name: torch
Version: 1.5.1+cu101
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, future
Required-by: torchvision, torchtext, fastai


In [ ]:
def load_data():
  adj = nx.adjacency_matrix(nx.read_edgelist("yeast.edgelist",
                                                   delimiter='\t',
                                                   create_using=nx.Graph()))
  features = sp.identity(adj.shape[0])
  features = torch.FloatTensor(np.array(features.todense()))

  return adj, features

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape


def mask_test_edges(adj):
    # Function to build test set with 2% positive links
    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
    adj.eliminate_zeros()

    adj_triu = sp.triu(adj)
    adj_tuple = sparse_to_tuple(adj_triu)
    edges = adj_tuple[0]
    edges_all = sparse_to_tuple(adj)[0]
    num_test = int(np.floor(edges.shape[0] / 50.))
    num_val = int(np.floor(edges.shape[0] / 50.))

    all_edge_idx = list(range(edges.shape[0]))
    np.random.shuffle(all_edge_idx)
    val_edge_idx = all_edge_idx[:num_val]
    test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
    test_edges = edges[test_edge_idx]
    val_edges = edges[val_edge_idx]
    train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

    def ismember(a, b):
        rows_close = np.all((a - b[:, None]) == 0, axis=-1)
        return np.any(rows_close)

    test_edges_false = []
    while len(test_edges_false) < len(test_edges):
        n_rnd = len(test_edges) - len(test_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], edges_all):
                continue
            if test_edges_false:
                if ismember([idx_j, idx_i], np.array(test_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(test_edges_false)):
                    continue
            test_edges_false.append([idx_i, idx_j])

    val_edges_false = []
    while len(val_edges_false) < len(val_edges):
        n_rnd = len(val_edges) - len(val_edges_false)
        rnd = np.random.randint(0, adj.shape[0], size=2 * n_rnd)
        idxs_i = rnd[:n_rnd]                                        
        idxs_j = rnd[n_rnd:]
        for i in range(n_rnd):
            idx_i = idxs_i[i]
            idx_j = idxs_j[i]
            if idx_i == idx_j:
                continue
            if ismember([idx_i, idx_j], train_edges):
                continue
            if ismember([idx_j, idx_i], train_edges):
                continue
            if ismember([idx_i, idx_j], val_edges):
                continue
            if ismember([idx_j, idx_i], val_edges):
                continue
            if val_edges_false:
                if ismember([idx_j, idx_i], np.array(val_edges_false)):
                    continue
                if ismember([idx_i, idx_j], np.array(val_edges_false)):
                    continue
            val_edges_false.append([idx_i, idx_j])

    # Re-build adj matrix
    data = np.ones(train_edges.shape[0])
    adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
    adj_train = adj_train + adj_train.T

    return adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false


def preprocess_graph(adj):
    adj = sp.coo_matrix(adj)
    adj_ = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj_.sum(1))
    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
    # return sparse_to_tuple(adj_normalized)
    return sparse_mx_to_torch_sparse_tensor(adj_normalized)


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


def get_roc_score(emb, adj_orig, edges_pos, edges_neg):
    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds_neg))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [ ]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, dropout=0., act=F.relu):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.act = act
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight)

    def forward(self, input, adj):
        input = F.dropout(input, self.dropout, self.training)
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        output = self.act(output)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
class GCNModelVAE(nn.Module):
    def __init__(self, input_feat_dim, hidden_dim1, hidden_dim2, dropout):
        super(GCNModelVAE, self).__init__()
        self.gc1 = GraphConvolution(input_feat_dim, hidden_dim1, dropout, act=F.relu)
        self.gc2 = GraphConvolution(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.gc3 = GraphConvolution(hidden_dim1, hidden_dim2, dropout, act=lambda x: x)
        self.dc = InnerProductDecoder(dropout, act=lambda x: x)

    def encode(self, x, adj):
        hidden1 = self.gc1(x, adj)
        return self.gc2(hidden1, adj), self.gc3(hidden1, adj)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(logvar)
            eps = torch.randn_like(std)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def forward(self, x, adj):
        mu, logvar = self.encode(x, adj)
        z = self.reparameterize(mu, logvar)
        return self.dc(z), mu, logvar


class InnerProductDecoder(nn.Module):
    """Decoder for using inner product for prediction."""

    def __init__(self, dropout, act=torch.sigmoid):
        super(InnerProductDecoder, self).__init__()
        self.dropout = dropout
        self.act = act

    def forward(self, z):
        z = F.dropout(z, self.dropout, training=self.training)
        adj = self.act(torch.mm(z, z.t()))
        return adj

In [ ]:
def loss_function(preds, labels, mu, logvar, n_nodes, norm, pos_weight):
    cost = norm * F.binary_cross_entropy_with_logits(preds, labels, pos_weight=pos_weight)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 / n_nodes * torch.mean(torch.sum(
        1 + 2 * logvar - mu.pow(2) - logvar.exp().pow(2), 1))
    return cost + KLD

In [ ]:
import sys
sys.argv=['']
del sys

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, default='gcn_vae', help="models used")
parser.add_argument('--seed', type=int, default=42, help='Random seed.')
parser.add_argument('--epochs', type=int, default=20, help='Number of epochs to train.')
parser.add_argument('--hidden1', type=int, default=32, help='Number of units in hidden layer 1.')
parser.add_argument('--hidden2', type=int, default=16, help='Number of units in hidden layer 2.')
parser.add_argument('--lr', type=float, default=0.01, help='Initial learning rate.')
parser.add_argument('--dropout', type=float, default=0., help='Dropout rate (1 - keep probability).')

args = parser.parse_args()

In [ ]:
adj, features = load_data()
n_nodes, feat_dim = features.shape

# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

In [ ]:
adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
adj = adj_train

In [ ]:
# Some preprocessing
adj_norm = preprocess_graph(adj)
adj_label = adj_train + sp.eye(adj_train.shape[0])
# adj_label = sparse_to_tuple(adj_label)
adj_label = torch.FloatTensor(adj_label.toarray())

pos_weight = torch.Tensor([float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()])
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

In [ ]:
model = GCNModelVAE(feat_dim, args.hidden1, args.hidden2, args.dropout)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

In [ ]:
hidden_emb = None

In [153]:
from sklearn.metrics import roc_auc_score, average_precision_score
for epoch in range(args.epochs):
        model = GCNModelVAE(feat_dim, args.hidden1, args.hidden2, args.dropout)
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
        hidden_emb = None
        t = time.time()
        model.train()
        optimizer.zero_grad()
        recovered, mu, logvar = model(features, adj_norm)
        loss = loss_function(preds=recovered, labels=adj_label,
                             mu=mu, logvar=logvar, n_nodes=n_nodes,
                             norm=norm, pos_weight=pos_weight)
        loss.backward()
        cur_loss = loss.item()
        optimizer.step()

        hidden_emb = mu.data.numpy()
        roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)

        print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss),
              "val_ap=", "{:.5f}".format(ap_curr),
              "time=", "{:.5f}".format(time.time() - t)
              )

print("Optimization Finished!")

roc_score, ap_score = get_roc_score(hidden_emb, adj_orig, test_edges, test_edges_false)
print('Test ROC score: ' + str(roc_score))
print('Test AP score: ' + str(ap_score))

Epoch: 0001 train_loss= 1.72352 val_ap= 0.63463 time= 1.79950
Epoch: 0002 train_loss= 1.73330 val_ap= 0.63891 time= 1.80722
Epoch: 0003 train_loss= 1.73539 val_ap= 0.66264 time= 1.79907
Epoch: 0004 train_loss= 1.71998 val_ap= 0.65141 time= 1.79061
Epoch: 0005 train_loss= 1.72761 val_ap= 0.67867 time= 1.80107
Epoch: 0006 train_loss= 1.72623 val_ap= 0.65130 time= 1.78900
Epoch: 0007 train_loss= 1.73250 val_ap= 0.65514 time= 1.79177
Epoch: 0008 train_loss= 1.73799 val_ap= 0.64825 time= 1.78052
Epoch: 0009 train_loss= 1.72583 val_ap= 0.65094 time= 1.81512
Epoch: 0010 train_loss= 1.72268 val_ap= 0.67309 time= 1.78764
Epoch: 0011 train_loss= 1.73906 val_ap= 0.64871 time= 1.78613
Epoch: 0012 train_loss= 1.72031 val_ap= 0.63863 time= 1.78544
Epoch: 0013 train_loss= 1.72186 val_ap= 0.65757 time= 1.80489
Epoch: 0014 train_loss= 1.71460 val_ap= 0.63926 time= 1.80908
Epoch: 0015 train_loss= 1.72983 val_ap= 0.66232 time= 1.81706
Epoch: 0016 train_loss= 1.74132 val_ap= 0.64449 time= 1.82356
Epoch: 0

In [ ]:
print("Loaded",adj.shape[0],"nodes")
print("Loaded",adj.sum(),"edges")
print()
print("-- Data format --")
print("Full graph adjacency shape:    ", adj.shape, "\t",             type(adj), "number of indices", len(adj.indices))
print("Training graph adjacency shape:", adj_train.shape, "\t",       type(adj_train), "number of indices", len(adj_train.indices))
print("val_edges:                     ", val_edges.shape, "\t",       type(val_edges))
print("val_edges_false:               ", len(val_edges_false), "\t\t",type(val_edges_false))
print("test_edges:                    ", test_edges.shape,"\t",       type(test_edges))
print("test_edges_false:              ", len(test_edges_false),"\t\t",type(test_edges_false))

Loaded 6526 nodes
Loaded 1062675 edges

-- Data format --
Full graph adjacency shape:     (6526, 6526) 	 <class 'scipy.sparse.csr.csr_matrix'> number of indices 1062675
Training graph adjacency shape: (6526, 6526) 	 <class 'scipy.sparse.csr.csr_matrix'> number of indices 1018554
val_edges:                      (10609, 2) 	 <class 'numpy.ndarray'>
val_edges_false:                10609 		 <class 'list'>
test_edges:                     (10609, 2) 	 <class 'numpy.ndarray'>
test_edges_false:               10609 		 <class 'list'>


In [ ]:
import itertools

hyperparams = {
    "hidden1": [256, 128, 64, 32],
    "hidden2": [64, 32, 16, 8],
    "lr": [0.001, 0.01, 0.1]}

keys = hyperparams.keys()
values = (hyperparams[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]
print("Total combinations:" ,len(combinations), ".")

Total combinations: 48 .


In [74]:
combinations[0]

{'hidden1': 256, 'hidden2': 64, 'lr': 0.001}

In [61]:
for key in hyperparams.keys():
  print(key)

hidden1
hidden2
lr


In [81]:
combinations[1]

{'hidden1': 256, 'hidden2': 64, 'lr': 0.01}

In [93]:
for d in combinations:
    print(d['hidden1'], d['hidden2'], d['lr'])

256 64 0.001
256 64 0.01
256 64 0.1
256 32 0.001
256 32 0.01
256 32 0.1
256 16 0.001
256 16 0.01
256 16 0.1
256 8 0.001
256 8 0.01
256 8 0.1
128 64 0.001
128 64 0.01
128 64 0.1
128 32 0.001
128 32 0.01
128 32 0.1
128 16 0.001
128 16 0.01
128 16 0.1
128 8 0.001
128 8 0.01
128 8 0.1
64 64 0.001
64 64 0.01
64 64 0.1
64 32 0.001
64 32 0.01
64 32 0.1
64 16 0.001
64 16 0.01
64 16 0.1
64 8 0.001
64 8 0.01
64 8 0.1
32 64 0.001
32 64 0.01
32 64 0.1
32 32 0.001
32 32 0.01
32 32 0.1
32 16 0.001
32 16 0.01
32 16 0.1
32 8 0.001
32 8 0.01
32 8 0.1


In [142]:
file1 = open("all_hyperparam_combinations.txt","w") 
for d in combinations:
  file1.write("hidenn1:{}, hidden2:{}, lr:{}\n".format(d['hidden1'], d['hidden2'], d['lr']))
file1.close()

In [152]:
file2 = open("20-epochs.txt","w") 

for d in combinations:
  file2.write("Settings: hidden1{}, hidden2{}, lr:{}\n".format(d['hidden1'], d['hidden2'], d['lr']))
  print("Settings:", d['hidden1'], d['hidden2'], d['lr'])
  for epoch in range(20): #5,10,15,20
    #print("Settings:", d['hidden1'], d['hidden2'], d['lr'])
    model = GCNModelVAE(feat_dim, d['hidden1'], d['hidden2'], args.dropout)
    optimizer = optim.Adam(model.parameters(), lr=d['lr'])
    hidden_emb = None
    t = time.time()
    model.train()
    optimizer.zero_grad()
    recovered, mu, logvar = model(features, adj_norm)
    loss = loss_function(preds=recovered, labels=adj_label, mu=mu, logvar=logvar, n_nodes=n_nodes, norm=norm, pos_weight=pos_weight)
    loss.backward()
    cur_loss = loss.item()
    optimizer.step()

    hidden_emb = mu.data.numpy()
    roc_curr, ap_curr = get_roc_score(hidden_emb, adj_orig, val_edges, val_edges_false)
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(cur_loss), "val_ap=", "{:.5f}".format(ap_curr), "time=", "{:.5f}".format(time.time() - t))
    file2.write("\n")
    file2.write("Epoch %04d" % (epoch + 1) )
    file2.write(" train loss= {:.5f}".format(cur_loss))
    file2.write(" val_ap= {:.5f}".format(ap_curr))
    file2.write(" time= {:.5f}".format(time.time() - t ))
    file2.write("\n")
    
  print("Optimization Finished!")
  file2.write("Optimization Finished! \n")
  roc_score, ap_score = get_roc_score(hidden_emb, adj_orig, test_edges, test_edges_false)
  print('Test ROC score: ' + str(roc_score))
  print('Test AP score: ' + str(ap_score))
  file2.write("Test ROC score: {}\n".format(str(roc_score)))
  file2.write("Test AP score: {}\n".format(str(ap_score)))
  file2.write("\n")

file2.close()

Settings: 256 64 0.001
Epoch: 0001 train_loss= 3.27924 val_ap= 0.65972 time= 2.47072
Epoch: 0002 train_loss= 3.25054 val_ap= 0.65873 time= 2.44192
Epoch: 0003 train_loss= 3.24807 val_ap= 0.64849 time= 2.45296
Epoch: 0004 train_loss= 3.25116 val_ap= 0.65905 time= 2.46442
Epoch: 0005 train_loss= 3.25636 val_ap= 0.65407 time= 2.44182
Epoch: 0006 train_loss= 3.26239 val_ap= 0.66069 time= 2.45104
Epoch: 0007 train_loss= 3.24739 val_ap= 0.65514 time= 2.44635
Epoch: 0008 train_loss= 3.25308 val_ap= 0.64955 time= 2.45240
Epoch: 0009 train_loss= 3.25037 val_ap= 0.65717 time= 2.45109
Epoch: 0010 train_loss= 3.24916 val_ap= 0.66028 time= 2.43430
Epoch: 0011 train_loss= 3.25476 val_ap= 0.65467 time= 2.45500
Epoch: 0012 train_loss= 3.26975 val_ap= 0.64997 time= 2.41686
Epoch: 0013 train_loss= 3.26772 val_ap= 0.64576 time= 2.43490
Epoch: 0014 train_loss= 3.26702 val_ap= 0.65169 time= 2.45300
Epoch: 0015 train_loss= 3.24487 val_ap= 0.65052 time= 2.49831
Epoch: 0016 train_loss= 3.26059 val_ap= 0.65142